# Exercise 2

You are given a collection of measurements of the form
$$ z = f \left( x, y \right) + \epsilon, $$
where $\epsilon$ is some random noise following the distribution 
$$\epsilon \sim \text{Laplace} \left( \mathbf{0},0.6\mathbf{I} \right).$$
Your goal is to recover the mapping $f(x,y)$ for all $x,y \in [-3,3]\times [-3,3]$. 

1. Start by implementing a simple NN with linear layers and try to recover $f(x,y)$ using the Mean Square Error (MSE) loss. The provided test set give a good approximation of the performance of your model. Note that, in practice, we would probably not be able to compute this approximation.

2. The MSE loss is not optimal for this problem. Why? What would be a better loss? Using the **same** network (but instantiate a new object), try optimizing with another (better) loss?

3. Look at the loss function of the training an the validation set. What can you deduct?

4. What happens if $\epsilon$ is Gaussian? Turn off the parameter `use_laplacian` and rerun the experiments.

What do you conclude from this exercise?


In [ ]:
# Autoreload is to always reload the imported python files.
%load_ext autoreload
%autoreload 2
# Matplotlib inline allows to make plot inline with the notebook with Matplotlib
%matplotlib inline

In [ ]:
# import all packages
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
import torch
import matplotlib.pyplot as plt
from torchsummary import summary
from poutyne import Model, SKLearnMetrics
from sklearn.metrics import r2_score, median_absolute_error
from utils import metric_flatten, get_poutyne_callbacks, saferm



In [ ]:
# These line allows to select the first GPU of the machine or the CPU is not GPU is present
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

## Dataset generation

In [ ]:
# Assume you don't know the data generating function.

def ground_truth(xx,yy):
    y = np.sin(yy*2) + np.cos(xx*3)+ - np.abs(xx-0.5) 
    return y

def noisy1(y):
    return y + 0.6*np.random.randn(*y.shape).astype(y.dtype)

def noisy2(y):
    return y + 0.6*np.random.laplace(size=y.shape).astype(y.dtype)


In [ ]:
lim = 3
x = np.arange(-lim, lim, 0.03)
y = np.arange(-lim, lim, 0.03)
xx, yy = np.meshgrid(x, y, sparse=False)
Z1 = ground_truth(xx,yy)
Z2 = noisy1(Z1)
Z3 = noisy2(Z1)

vmin = np.min(Z1)
vmax = np.max(Z1)
plt.figure(figsize=(10, 3))
plt.subplot(1,3,1)
plt.imshow(Z1, vmin=vmin, vmax=vmax, extent=[-lim,lim,-lim,lim])
plt.colorbar()
plt.axis('scaled')
plt.title("Ground truth")

plt.subplot(1,3,2)
plt.imshow(Z2, vmin=vmin, vmax=vmax, extent=[-lim,lim,-lim,lim])
plt.colorbar()
plt.axis('scaled')
plt.title("Gaussian noise")

plt.subplot(1,3,3)
plt.imshow(Z3, vmin=vmin, vmax=vmax, extent=[-lim,lim,-lim,lim])
plt.colorbar()
plt.axis('scaled')
plt.title("Laplacian noise")


In [ ]:
## dataset creation
def sample_generator(n, no_noise=False, laplacian=False):
    # Random sampling of points
    if no_noise:
        xx = np.arange(-lim, lim, 2*lim/np.sqrt(n))
        yy = np.arange(-lim, lim, 2*lim/np.sqrt(n))
        x = np.array(np.meshgrid(xx, yy, sparse=False)).reshape(2,-1).T.astype(np.float32)
    else:
        np.random.seed(0)
        x = np.random.rand(n, 2).astype(np.float32)*2*lim -lim
    # compute the output
    # we expand the dimension to have the size [n x 1], which will be the output shape of the NN.
    y = np.expand_dims(ground_truth(x[:,0], x[:,1]),axis=1)
    if not(no_noise):
        if laplacian:
            y = noisy2(y)
        else:
            y = noisy1(y)
    # Convertion to Pytorch tensors
    return torch.tensor(x), torch.tensor(y)

n_train = 1024

use_laplacian = True

train_data = sample_generator(n_train, laplacian=use_laplacian)
valid_data = sample_generator(64, laplacian=use_laplacian)
test_data = sample_generator(256, no_noise=True, laplacian=use_laplacian)


In [ ]:
plt.scatter(train_data[0][:,0], train_data[0][:,1], c=train_data[1][:,0])

In [ ]:
# It is very practical to transform our data into torch dataset
train_dataset = TensorDataset(*train_data)
valid_dataset = TensorDataset(*valid_data)
test_dataset = TensorDataset(*test_data)

## Task 1: Find a mapping using the MSE loss

In [ ]:
# Define your model here


Train the model

Evaluate the model

We can compute the score on the test set.


We can also perform prediction on a grid of points an look at the results.

## Train the same model using a different loss
Do not forget to create a new object for the network and the model.

Compare the result